In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
from data_prep import Password as P
import logging

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

USE_CUDA = True
PRE_GEN_ITERS = 100
ADVERSIVE_ITERS = 500
CRITIC_ITERS = 1
BATCHSIZE = 1

device = torch.device("cuda" if torch.cuda.is_available() and USE_CUDA else "cpu")

In [2]:
# custom weights initialization called on G and D
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        nn.init.normal_(m.weight.data, 0, 0.001)

In [3]:
class Discriminator(nn.Module):
    def __init__(self, hidden_size):
        super(Discriminator, self).__init__()

        self.input_size = P.n_letters + 1
        self.hidden_size = hidden_size
        self.output_size = 1
        
        #self.gru = nn.GRU(self.input_size, self.hidden_size, self.layers)
        #self.h2o = nn.Linear(self.hidden_size, self.output_size)
        #self.hidden = self.initHidden()
        
        self.i2h = nn.Sequential(*[
            nn.Linear(self.input_size + self.hidden_size, self.hidden_size)
        ])
        self.i2o = nn.Sequential(*[
            nn.Linear(self.input_size + self.hidden_size, self.output_size),
        ])
        self.i2h.apply(weights_init)
        self.i2o.apply(weights_init)
        
        
    def forward(self, input, hidden):
        #output, hidden = self.gru(input.view(1,1,-1), self.hidden)
        #output = self.h2o(output)
        
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size).to(device)
    
    def requiresGrad(self):
        for p in self.parameters():
            p.requires_grad = True
            
    def requiresNoGrad(self):
        for p in self.parameters():
            p.requires_grad = False
            
    def calcGradientPenalty(self, real_data, fake_data, LAMBDA = .5):
        if real_data.size()[0] > fake_data.size()[0]:
            fake_data = P.zeroPadding(fake_data.to(torch.device("cpu")), real_data.size(0)).to(device)
        elif real_data.size()[0] < fake_data.size()[0]:
            real_data = P.zeroPadding(real_data.to(torch.device("cpu")), fake_data.size(0)).to(device)
        alpha = torch.rand(1).to(device)
        interpolate = alpha * real_data + (1 - alpha) * fake_data
        interpolate.requires_grad = True

        output = self.discriminate(interpolate)

        grad = torch.ones_like(interpolate).to(device)
        gradients = torch.autograd.grad(
                outputs=output,
                inputs=interpolate,
                grad_outputs=grad,
                create_graph=True,
                retain_graph=True,
                only_inputs=True,
            )[0]
        # logging.debug(gradients.norm(2, dim=2))
        penalty = ((gradients.norm(2, dim=2) - 1 ) ** 2).mean() * LAMBDA
        return penalty
    
    def train(self, input_tensor, category):
        hidden = self.initHidden()
        self.zero_grad()
        
        for i in range(input_tensor.size()[0]):
            output, hidden = self(input_tensor[i], hidden)
            
        loss = -output * category
        # loss = loss * torch.sigmoid(100*loss)
        loss.backward(retain_graph = True)

        return output, loss.item()
    
    def discriminate(self, input_tensor):
        hidden = self.initHidden()
        for i in range(input_tensor.size()[0]):
            output, hidden = self(input_tensor[i], hidden)
        return output

In [4]:
class Generator(nn.Module):
    def __init__(self, hidden_size):
        super(Generator, self).__init__()
        
        self.input_size = P.n_letters + 1
        self.hidden_size = hidden_size
        self.output_size = P.n_letters + 1
        self.layers = 2
        
        self.gru = nn.GRU(self.input_size, self.hidden_size, self.layers)
        #self.lstm = nn.LSTM(self.input_size, self.hidden_size)
        self.h2o = nn.Linear(self.hidden_size, self.output_size)
        self.dropout = nn.Dropout(0.1)
        self.hidden = self.initHiddenZeros()

    def forward(self, input):
        output, self.hidden = self.gru(input.view(1,1,-1), self.hidden)
        output = self.h2o(output)
        output = self.dropout(output)
        output = F.log_softmax(output,dim=2)
        return output

    def initHiddenZeros(self):
        return torch.zeros(self.layers, 1, self.hidden_size).to(device)
    
    def initHiddenRand(self):
        return torch.rand(self.layers,1, self.hidden_size).to(device)
    
    def generatePassTensor(self, max_length = 18):
        start_letter = p.passwords_string[random.randint(0,len(p.passwords_string) - 1)][0]
        with torch.no_grad():
            input_tensor = P.passwordToInputTensor(start_letter).to(device)
            self.hidden = self.initHiddenZeros()
            password = start_letter

            for c in range(max_length):
                output = self(input_tensor[0])
                output = output.view(1,-1)
                topv, topi = output.topk(1)
                topi = topi[0][0]
                if topi == P.n_letters - 1:
                    break
                else:
                    letter = P.all_letters[topi]
                    password += letter
                input_tensor = P.passwordToInputTensor(letter).to(device)
        
        return P.passwordToInputTensor(password)
        
    
    def generate_N(self, p, n_generate = 100, max_length = 18):
        generate_list = []

        for i in range(n_generate):
            start_letter = p.passwords_string[random.randint(0,len(p.passwords_string) - 1)][0]
            with torch.no_grad():
                input_tensor = P.passwordToInputTensor(start_letter).to(device)
                self.hidden = self.initHiddenZeros()
                output_password = start_letter

                for c in range(max_length):
                    output = self(input_tensor[0])
                    output = output.view(1,-1)
                    topv, topi = output.topk(1)
                    topi = topi[0][0]
                    if topi == P.n_letters - 1:
                        break
                    else:
                        letter = P.all_letters[topi]
                        output_password += letter
                    input_tensor = P.passwordToInputTensor(letter).to(device)
                    
            generate_list.append(output_password)
            
        return generate_list
    
    
    def gen_from_TH(self, input_real_tensor):
        self.hidden = self.initHiddenZeros()
        
        
        
        

    
    def pre_train(self, input_line_tensor, target_line_tensor):
        target_line_tensor.unsqueeze_(-1)
        self.hidden = self.initHiddenZeros()

        criterion = nn.NLLLoss()
        optimizer = torch.optim.SGD(self.parameters(), lr=0.1)

        self.zero_grad()
        loss = torch.tensor(0, dtype = torch.float32, requires_grad = True, device = device)

        for i in range(input_line_tensor.size(0)):
            output = self(input_line_tensor[i])
            output = output.view(1,-1)
            l = criterion(output, target_line_tensor[i])
            loss = loss + l

        loss.backward()
        optimizer.step()

        return output, loss.item() / input_line_tensor.size(0)
    
    
    def train(self, D, p, criterion = nn.NLLLoss(), learning_rate = 0.005, max_length = 18):
        start_letter = p.passwords_string[random.randint(0,len(p.passwords_string) - 1)][0]
        start_tensor = P.passwordToInputTensor(start_letter).to(device)
        output_tensor = start_tensor
        self.hidden = self.initHiddenZeros()
       
        output = start_tensor.to(device)
        for c in range(max_length):
            output = self(output[0])
            output_tensor = torch.cat((output_tensor, torch.exp(output)), -3)
            topv, topi = output.topk(1)
            topi = topi[0][0]
            if topi == P.n_letters - 1:
                break
            
        output = D.discriminate(output_tensor)
        target_tensor = torch.tensor([1], dtype = torch.long, device = device)
        loss = -output
        logging.debug("gen loss: "+str(loss.item()))
        #loss = criterion(output, target_tensor)
        loss.backward()
        
        for p in self.parameters():
            p.data.add_(-learning_rate, p.grad.data)

In [5]:
g = Generator(512).to(device)
d = Discriminator(1024).to(device)

In [6]:
#opt_g = torch.optim.Adam(g.parameters(), lr=0.0002, betas=(0.5, 0.999))
#opt_d = torch.optim.Adam(d.parameters(), lr=0.0002, betas=(0.5, 0.999))
opt_g = torch.optim.RMSprop(g.parameters(), lr=0.0002)
opt_d = torch.optim.RMSprop(d.parameters(), lr=0.0002)

In [7]:
p = P()
passwords = p.passIter()

INFO:root:Initializing passwords...
DEBUG:root:Loading from existing json file...
INFO:root:Done initializing passwords.


In [8]:
print_every = 100
loss_trend = []
logging.info("---------- Pre-training generator ----------")
for i in range(PRE_GEN_ITERS):
    pas = next(passwords)
    input_tensor = P.passwordToInputTensor(pas).to(device)
    target_tensor = P.passwordToTargetTensor(pas).to(device)
    output, loss = g.pre_train(input_tensor, target_tensor)
    
    if i % print_every == 0:
        loss_trend.append(loss)
        logging.debug("Iter: "+ str(i)+" Loss: "+str(loss))

INFO:root:---------- Pre-training generator ----------
DEBUG:root:Iter: 0 Loss: 4.399033228556315


In [9]:
logger.setLevel(logging.DEBUG)
logging.info("---------- Adversarial Training ----------\n")

for i in range(1, ADVERSIVE_ITERS + 1):
    logging.debug("----------------- %d / %d -----------------" % (i, ADVERSIVE_ITERS))
    logging.debug("Training discriminator...")
    
    d.requiresGrad()
    g.zero_grad()
    
    correct = 0
    for j in range(CRITIC_ITERS):
        L = 0
        # Genuine
        thisPass = next(passwords)
        logging.debug("real password is: "+thisPass)
        genuine_password_tensor = P.passwordToInputTensor(thisPass).to(device)
        category = 1
        output, loss_real = d.train(genuine_password_tensor, category)
        logging.debug("real loss: "+str(loss_real))
        L += loss_real
        if output > 0:
            correct = correct + 1
            
        # Fake
        fake_password_tensor = g.generatePassTensor().to(device)
        logging.debug("fake password length: "+str(fake_password_tensor.size()[0]))
        category = -1
        output, loss_fake = d.train(fake_password_tensor, category)
        logging.debug("fake loss: "+str(loss_fake))
        L += loss_fake
        if output < 0:
            correct = correct + 1
            
        # Gradient penalty
        loss_gp = d.calcGradientPenalty(genuine_password_tensor, fake_password_tensor, 0.02)
        loss_gp.backward(retain_graph=False)
        logging.debug("g:"+str(loss_gp.item()))
        L += loss_gp
        
        opt_d.step()
        
    logging.debug("Done. Classification accuracy: " + str(correct / CRITIC_ITERS / 2) + " Loss: " + str(L.item()))    
    
    logging.debug("Training generator...")
    
    d.requiresNoGrad()
    
    g.train(d, p)
    
    logging.debug("Done training generator.\n")
    

INFO:root:---------- Adversarial Training ----------

DEBUG:root:----------------- 1 / 500 -----------------
DEBUG:root:Training discriminator...
DEBUG:root:real password is: alexandra
DEBUG:root:real loss: 0.008741972036659718
DEBUG:root:fake password length: 5
DEBUG:root:fake loss: -0.007819322869181633
DEBUG:root:g:0.0748773068189621
DEBUG:root:Done. Classification accuracy: 0.5 Loss: 0.07579995691776276
DEBUG:root:Training generator...
DEBUG:root:gen loss: 0.07079263031482697
DEBUG:root:Done training generator.

DEBUG:root:----------------- 2 / 500 -----------------
DEBUG:root:Training discriminator...
DEBUG:root:real password is: alexis
DEBUG:root:real loss: 0.07070823013782501
DEBUG:root:fake password length: 7
DEBUG:root:fake loss: -0.07159435749053955
DEBUG:root:g:0.29026100039482117
DEBUG:root:Done. Classification accuracy: 0.5 Loss: 0.2893748879432678
DEBUG:root:Training generator...
DEBUG:root:gen loss: 0.31818827986717224
DEBUG:root:Done training generator.

DEBUG:root:----

DEBUG:root:g:0.12359130382537842
DEBUG:root:Done. Classification accuracy: 0.5 Loss: 0.12092030048370361
DEBUG:root:Training generator...
DEBUG:root:gen loss: 0.03404244780540466
DEBUG:root:Done training generator.

DEBUG:root:----------------- 19 / 500 -----------------
DEBUG:root:Training discriminator...
DEBUG:root:real password is: 121212
DEBUG:root:real loss: 0.03250490874052048
DEBUG:root:fake password length: 3
DEBUG:root:fake loss: -0.03355543315410614
DEBUG:root:g:0.06559211015701294
DEBUG:root:Done. Classification accuracy: 0.5 Loss: 0.06454158574342728
DEBUG:root:Training generator...
DEBUG:root:gen loss: 0.0332290455698967
DEBUG:root:Done training generator.

DEBUG:root:----------------- 20 / 500 -----------------
DEBUG:root:Training discriminator...
DEBUG:root:real password is: sayang
DEBUG:root:real loss: 0.032253965735435486
DEBUG:root:fake password length: 7
DEBUG:root:fake loss: -0.03664311021566391
DEBUG:root:g:0.07936521619558334
DEBUG:root:Done. Classification accur

DEBUG:root:Training discriminator...
DEBUG:root:real password is: shorty
DEBUG:root:real loss: 0.03599017858505249
DEBUG:root:fake password length: 7
DEBUG:root:fake loss: -0.03902843967080116
DEBUG:root:g:0.051884591579437256
DEBUG:root:Done. Classification accuracy: 0.5 Loss: 0.04884633049368858
DEBUG:root:Training generator...
DEBUG:root:gen loss: 0.037096597254276276
DEBUG:root:Done training generator.

DEBUG:root:----------------- 37 / 500 -----------------
DEBUG:root:Training discriminator...
DEBUG:root:real password is: 11111
DEBUG:root:real loss: 0.03482038155198097
DEBUG:root:fake password length: 6
DEBUG:root:fake loss: -0.03889717534184456
DEBUG:root:g:0.03934731334447861
DEBUG:root:Done. Classification accuracy: 0.5 Loss: 0.03527051955461502
DEBUG:root:Training generator...
DEBUG:root:gen loss: 0.0381142757833004
DEBUG:root:Done training generator.

DEBUG:root:----------------- 38 / 500 -----------------
DEBUG:root:Training discriminator...
DEBUG:root:real password is: nath

DEBUG:root:Training generator...
DEBUG:root:gen loss: 0.04494980722665787
DEBUG:root:Done training generator.

DEBUG:root:----------------- 54 / 500 -----------------
DEBUG:root:Training discriminator...
DEBUG:root:real password is: cheese
DEBUG:root:real loss: 0.046514950692653656
DEBUG:root:fake password length: 7
DEBUG:root:fake loss: -0.045804463326931
DEBUG:root:g:0.035705938935279846
DEBUG:root:Done. Classification accuracy: 0.5 Loss: 0.0364164263010025
DEBUG:root:Training generator...
DEBUG:root:gen loss: 0.04509571194648743
DEBUG:root:Done training generator.

DEBUG:root:----------------- 55 / 500 -----------------
DEBUG:root:Training discriminator...
DEBUG:root:real password is: 159753
DEBUG:root:real loss: 0.04410406947135925
DEBUG:root:fake password length: 6
DEBUG:root:fake loss: -0.04792151227593422
DEBUG:root:g:0.02649104967713356
DEBUG:root:Done. Classification accuracy: 0.5 Loss: 0.022673606872558594
DEBUG:root:Training generator...
DEBUG:root:gen loss: 0.04620592296123

DEBUG:root:fake password length: 5
DEBUG:root:fake loss: -0.052051298320293427
DEBUG:root:g:0.01302674226462841
DEBUG:root:Done. Classification accuracy: 0.5 Loss: 0.010035628452897072
DEBUG:root:Training generator...
DEBUG:root:gen loss: 0.05095738545060158
DEBUG:root:Done training generator.

DEBUG:root:----------------- 72 / 500 -----------------
DEBUG:root:Training discriminator...
DEBUG:root:real password is: fernando
DEBUG:root:real loss: 0.053326450288295746
DEBUG:root:fake password length: 7
DEBUG:root:fake loss: -0.05344394966959953
DEBUG:root:g:0.03358837217092514
DEBUG:root:Done. Classification accuracy: 0.5 Loss: 0.03347087278962135
DEBUG:root:Training generator...
DEBUG:root:gen loss: 0.05253596603870392
DEBUG:root:Done training generator.

DEBUG:root:----------------- 73 / 500 -----------------
DEBUG:root:Training discriminator...
DEBUG:root:real password is: pokemon
DEBUG:root:real loss: 0.05226237326860428
DEBUG:root:fake password length: 7
DEBUG:root:fake loss: -0.0533

...

DEBUG:root:Done. Classification accuracy: 0.5 Loss: 0.07170349359512329
DEBUG:root:Training generator...
DEBUG:root:gen loss: 0.21115148067474365
DEBUG:root:Done training generator.

DEBUG:root:----------------- 423 / 500 -----------------
DEBUG:root:Training discriminator...
DEBUG:root:real password is: disney
DEBUG:root:real loss: 0.20532912015914917
DEBUG:root:fake password length: 7
DEBUG:root:fake loss: -0.1897408813238144
DEBUG:root:g:0.04930643364787102
DEBUG:root:Done. Classification accuracy: 0.5 Loss: 0.0648946762084961
DEBUG:root:Training generator...
DEBUG:root:gen loss: 0.21161751449108124
DEBUG:root:Done training generator.

DEBUG:root:----------------- 424 / 500 -----------------
DEBUG:root:Training discriminator...
DEBUG:root:real password is: rabbit
DEBUG:root:real loss: 0.20628508925437927
DEBUG:root:fake password length: 5
DEBUG:root:fake loss: -0.20127350091934204
DEBUG:root:g:0.03370034322142601
DEBUG:root:Done. Classification accuracy: 0.5 Loss: 0.0387119315564632

DEBUG:root:real password is: sporting
DEBUG:root:real loss: 0.2084028124809265
DEBUG:root:fake password length: 6
DEBUG:root:fake loss: -0.2145337462425232
DEBUG:root:g:0.023503176867961884
DEBUG:root:Done. Classification accuracy: 0.5 Loss: 0.017372243106365204
DEBUG:root:Training generator...
DEBUG:root:gen loss: 0.22436043620109558
DEBUG:root:Done training generator.

DEBUG:root:----------------- 441 / 500 -----------------
DEBUG:root:Training discriminator...
DEBUG:root:real password is: miranda
DEBUG:root:real loss: 0.2049146592617035
DEBUG:root:fake password length: 7
DEBUG:root:fake loss: -0.20691201090812683
DEBUG:root:g:0.006894953548908234
DEBUG:root:Done. Classification accuracy: 0.5 Loss: 0.004897601902484894
DEBUG:root:Training generator...
DEBUG:root:gen loss: 0.21958336234092712
DEBUG:root:Done training generator.

DEBUG:root:----------------- 442 / 500 -----------------
DEBUG:root:Training discriminator...
DEBUG:root:real password is: dallas
DEBUG:root:real loss: 0.2123

DEBUG:root:Training generator...
DEBUG:root:gen loss: 0.22078336775302887
DEBUG:root:Done training generator.

DEBUG:root:----------------- 458 / 500 -----------------
DEBUG:root:Training discriminator...
DEBUG:root:real password is: monster
DEBUG:root:real loss: 0.21800291538238525
DEBUG:root:fake password length: 5
DEBUG:root:fake loss: -0.21258807182312012
DEBUG:root:g:0.004111809190362692
DEBUG:root:Done. Classification accuracy: 0.5 Loss: 0.009526653215289116
DEBUG:root:Training generator...
DEBUG:root:gen loss: 0.2252899706363678
DEBUG:root:Done training generator.

DEBUG:root:----------------- 459 / 500 -----------------
DEBUG:root:Training discriminator...
DEBUG:root:real password is: player
DEBUG:root:real loss: 0.22220295667648315
DEBUG:root:fake password length: 7
DEBUG:root:fake loss: -0.2192443609237671
DEBUG:root:g:0.004146579187363386
DEBUG:root:Done. Classification accuracy: 0.5 Loss: 0.007105174940079451
DEBUG:root:Training generator...
DEBUG:root:gen loss: 0.233730614

DEBUG:root:real loss: 0.23097914457321167
DEBUG:root:fake password length: 3
DEBUG:root:fake loss: -0.1805163025856018
DEBUG:root:g:0.001706767943687737
DEBUG:root:Done. Classification accuracy: 0.5 Loss: 0.05216960981488228
DEBUG:root:Training generator...
DEBUG:root:gen loss: 0.23524729907512665
DEBUG:root:Done training generator.

DEBUG:root:----------------- 476 / 500 -----------------
DEBUG:root:Training discriminator...
DEBUG:root:real password is: parola
DEBUG:root:real loss: 0.23079398274421692
DEBUG:root:fake password length: 7
DEBUG:root:fake loss: -0.23244471848011017
DEBUG:root:g:0.003947827033698559
DEBUG:root:Done. Classification accuracy: 0.5 Loss: 0.0022970912978053093
DEBUG:root:Training generator...
DEBUG:root:gen loss: 0.24527491629123688
DEBUG:root:Done training generator.

DEBUG:root:----------------- 477 / 500 -----------------
DEBUG:root:Training discriminator...
DEBUG:root:real password is: evelyn
DEBUG:root:real loss: 0.23140329122543335
DEBUG:root:fake passwor

DEBUG:root:gen loss: 0.2574697732925415
DEBUG:root:Done training generator.

DEBUG:root:----------------- 493 / 500 -----------------
DEBUG:root:Training discriminator...
DEBUG:root:real password is: 999999999
DEBUG:root:real loss: 0.2550663352012634
DEBUG:root:fake password length: 7
DEBUG:root:fake loss: -0.24558526277542114
DEBUG:root:g:0.009729849174618721
DEBUG:root:Done. Classification accuracy: 0.5 Loss: 0.019210921600461006
DEBUG:root:Training generator...
DEBUG:root:gen loss: 0.25876128673553467
DEBUG:root:Done training generator.

DEBUG:root:----------------- 494 / 500 -----------------
DEBUG:root:Training discriminator...
DEBUG:root:real password is: broken
DEBUG:root:real loss: 0.2424958348274231
DEBUG:root:fake password length: 7
DEBUG:root:fake loss: -0.2462194710969925
DEBUG:root:g:0.003927573561668396
DEBUG:root:Done. Classification accuracy: 0.5 Loss: 0.00020393729209899902
DEBUG:root:Training generator...
DEBUG:root:gen loss: 0.25033992528915405
DEBUG:root:Done traini

In [18]:
g.generate_N(p, 5)

['4aaeeeeeeeaeeeeeeee',
 '6aaeeeeeeeeeeeaeeee',
 '3aeeeeaeeeeeeeeeeea',
 'meaeeeeeaeeeeeeeeee',
 'laaaeeaeeaeeaeeeeee']

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

%matplotlib inline

logger.setLevel(logging.INFO)

plt.figure()
plt.plot(loss_trend)
#plt.savefig('pictures/rnn-rnn-loss.png',dpi=400)
plt.show()

In [ ]:
m = nn.LogSoftmax(dim=1)
loss = nn.NLLLoss()
# input is of size N x C = 3 x 5
input = torch.randn(3, 5, requires_grad=True)
# each element in target has to have 0 <= value < C
target = torch.tensor([1, 0, 4])
output = loss(m(input), target)

In [ ]:
import torch
a = torch.ones(3,2)
a.requires_grad = True
b = a * 2
ones = torch.ones_like(a)
g = torch.autograd.grad(
            outputs=b,
            inputs=a,
            grad_outputs=ones,
            create_graph=True,
            retain_graph=True,
            only_inputs=True,
        )

In [ ]:
g

In [ ]:
g[0].view(g[0].size(0), -1).pow(2).sum(1)**(3)

In [38]:
torch.ones(1,1,2).norm(2, dim=2)[0][0]

tensor(True)

In [30]:
g[0]

TypeError: 'Generator' object is not subscriptable

In [ ]:
a = [0,1,2]
a[-1]

In [16]:
ADVERSIVE_ITERS = 200